<a href="https://colab.research.google.com/github/natalievolk/LearnAI/blob/main/LearnAI_Take_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import cv2
import requests
from io import BytesIO

In [49]:
train_txt = "https://raw.githubusercontent.com/natalievolk/LearnAI/training/train.txt"
test_txt = "https://raw.githubusercontent.com/natalievolk/LearnAI/training/test.txt"

In [50]:
class ImagesDataset(Dataset):
    def __init__(self, text_file, transform):
        self.df = pd.read_csv(text_file, sep=' ')
        self.df.columns = ['jpg_url', 'classification']
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        response = requests.get(self.df['jpg_url'][idx])
        img = Image.open(BytesIO(response.content)).resize((400, 400))
        pix = np.array(img)
        try:
          pix = self.transform(pix)
        # if image only has one channel because it's B&W
        except:
          print(self.df['jpg_url'][idx])
          # copy grayscale values into RGB values (because R=G=B=grayscale for B&W)
          pix = np.stack((np.array(pix, copy=True), np.array(pix, copy=True), np.array(pix, copy=True)), axis=2)
          pix = self.transform(pix)
          print("B&W dealt with :)")
 
        return pix, self.df['classification'][idx]

In [51]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = ImagesDataset(train_txt,transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, num_workers=8)

test_dataset = ImagesDataset(test_txt,transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, num_workers=8)

In [52]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5,2)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,12,7,2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(6348, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)

    # Defining the forward pass    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [53]:
net = Net()
optimizer = Adam(net.parameters(), lr=0.07)
criterion = CrossEntropyLoss()
if torch.cuda.is_available():
    net = net.cuda()
    criterion = criterion.cuda()
device = torch.device('cuda:0')
  

In [54]:
#Training Stage
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    print('[%d] loss: %f' %
              (epoch + 1, running_loss))

print('Finished Training')

http://ecx.images-amazon.com/images/I/41jM4XskO2L.jpg
B&W dealt with :)
http://ecx.images-amazon.com/images/I/31kwjAdxKcL.jpg
B&W dealt with :)
http://ecx.images-amazon.com/images/I/51mbrK7M1kL.jpg


KeyboardInterrupt: ignored

In [ ]:
dataiter = iter(train_dataloader)
images, labels = dataiter.next()

conv1 = nn.Conv2d(3,6,5,2)
pool = nn.MaxPool2d(2,2)
conv2 = nn.Conv2d(6,12,7,2)

x = conv1(images)
# print(x.shape)
x = pool(x)
# print(x.shape)
x = conv2(x)
print(x.shape)